# Load Dataset

https://huggingface.co/datasets/doc2dial#data-fields


In [ ]:
from datasets import load_dataset

split = "train"
cache_dir = "./data_cache_doc2vec_notebook"

#User's turn: utterance= question, reference=grounding document span_id, can be empty, "precondition"/"solution" are the
#actual grounding spans
#Gold label for grounding
dialogue_dataset = load_dataset(
    "doc2dial",
    name="dialogue_domain",  # this is the name of the dataset for the second subtask, dialog generation
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

document_dataset = load_dataset(
    "doc2dial",
    name="document_domain",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

Insights from looking at the data structure:
For training I will need the following data:
From Dialogue:
doc_id - Grounding document
turns.utterance -> Dialogue history
references.sp_id -> ID for the output text_sp

From Documents:
To get output span:
doc_id -> spans -> id_sp -> text_sp column is the actual output span


# Embedding with Gensim dense vectors

Hypothesis:
1. Create embeddings for each span in each doc
2. Create embeddings for each span combined with the user's question
    - think about if the agent's span should also be included

Further:
- think about training on a whole domain vs one document

In [2]:
def span_texts_for_doc(doc_id):
    document = document_dataset.filter(lambda doc: doc['doc_id'] == doc_id)
    return [pd.json_normalize(span) for span in document['spans']][0]['text_sp']

In [3]:
from gensim.utils import tokenize
from gensim.utils import simple_preprocess
#Getting a list of spans per grounding document !!!!Confusion spans are the documents and the document is the corpora in gensim language
#I'm only using documents that have been referenced to in the dialogue (which not all have) but for the further work I want user utterances which I won't have for some documents
import pandas as pd

docids_set = set(dialogue_dataset['doc_id'])
docids = list(docids_set)

#extract all the span texts for that doc id, !!!Index 0 is equal to sp_id = '1' and so forth!!!
#doc ids constantly change
# doc_id = 'Learn about personalized plates#3_0'
doc_id = 'Co-sign Your Spouse\'s Income-Driven Repayment Plan Application | Federal Student Aid#1_0'
spans = span_texts_for_doc(doc_id)

#this interprets hyphenated words with spaces as two words
#TODO there is quite a few variations of how to tokenize and how to preprocess the text which might make quite a big difference
#TODO try stemming and lemmatisation
tokenized_spans_simple = [list(tokenize(span, lower=True)) for span in spans]
print(tokenized_spans_simple)
#process spans and remove unhelpful words, simple preprocess seems to do a good general job
tokenized_spans_preprocessed = [list(simple_preprocess(span, deacc=True)) for span in spans]
print(tokenized_spans_preprocessed)
#TODO compare how the strings are different -> one thing I noticed is that there are not 's' as words hanging about in the simple preprocessing



  0%|          | 0/1 [00:00<?, ?ba/s]

[['what', 'is', 'a', 'co', 'signer'], ['a', 'co', 'signer', 'is', 'the', 'spouse', 'of', 'an', 'applicant', 'who', 'initiated', 'an', 'income', 'driven', 'repayment', 'plan', 'request'], ['as', 'a', 'co', 'signer', 'you', 'are', 'not', 'obligated', 'to', 'repay', 'this', 'loan', 'by', 'signing', 'a', 'borrower', 's', 'idr', 'application'], ['the', 'idr', 'applicant', 'should', 'have', 'provided', 'you', 'with', 'a', 'reference', 'number', 'co', 'sign', 'code'], ['if', 'you', 'do', 'not', 'have', 'the', 'reference', 'number', 'co', 'sign', 'code'], ['contact', 'the', 'idr', 'applicant'], ['an', 'e', 'mail', 'containing', 'the', 'reference', 'number', 'co', 'sign', 'code', 'was', 'sent', 'to', 'him', 'or', 'her'], ['the', 'idr', 'applicant', 'can', 'also', 'access', 'the', 'reference', 'number', 'co', 'sign', 'code', 'by', 'logging', 'in', 'to', 'studentaid', 'gov', 'and', 'clicking', 'on', 'the', 'appropriate', 'link'], ['directions', 'to', 'access', 'the', 'reference', 'number', 'co', 

# Using Doc2Vec model
to create a vector per doc (which is a span) given that I need to predict which spans are most relevant
this seems better
see https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py
https://radimrehurek.com/gensim/models/doc2vec.html

plan is to train a model on each document, first without adding the user's utterances to the span

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tokenized_spans = tokenized_spans_preprocessed
#tag is re-assigning an id to  each doc. I decided to use i+1 to match the tag the span has in the dataset
#TODO check if I can add the docid to the tags and train  on all documents from the same domain
training_docs = [TaggedDocument(doc, [i + 1]) for i, doc in enumerate(tokenized_spans)]
# Check how to fine tuen the model https://radimrehurek.com/gensim/models/doc2vec.html
model = Doc2Vec(training_docs, vector_size=10, window=4, min_count=1, workers=4, epochs=30)

#TODO save models I like to disk (although this was amazingly quick)

### Predict what span are grounding for the agents answer

**Question**
Do we need to predict the span that matches the users question? Or the spans that then are in the agents response
I believe it's the later or probably all

*I'm building my dataset adding spans used for the users question and the spans for the agent to the solution*

**Question**
Given that the dialogue dataset is not used for training, I'm assuming I can use it to see how this model performs. However as soon as I start to combine the user utterances to the document span than this will no longer be the case and I will have to use the validation test for evaluation.

In [6]:
import pandas as pd

dialogue_full_df = pd.DataFrame(data=dialogue_dataset)

In [7]:
#Code to get the dialogues for a doc id from the dialog set to do some manual inspections
dialogue_eval_df = pd.DataFrame(columns=['doc_id', 'dial_id', 'rc_id', 'turn_id', 'user_utterance', 'user_sp_ids',
                                         'agent_sp_ids'])

#using df as dataset is way too slow to do this
dialogues_for_doc = dialogue_full_df.loc[dialogue_full_df['doc_id'] == doc_id]

#resetting all variables
user_turn_index = 0
user_utterance = ''
user_sps = []
turn_id = ''
agent_sps = []
role = ''
dial_id = ''
rc_id = ''
first_user = True
#for all list of turns in each dialogue
for dialogue_index, turns in enumerate(dialogues_for_doc['turns']):

    #for each turn in a dialogue
    for turn in turns:
        role = turn['role']

        # for the very first user role only
        if role == 'user' and first_user == True:
            turn_id = turn['turn_id']
            user_utterance =turn['utterance']
            user_sps = [ref['sp_id'] for ref in turn['references']]
            dial_id = dialogues_for_doc.iloc[dialogue_index]['dial_id']
            rc_id = dial_id + '_'+ str(turn_id)
            first_user = False
        elif role == 'user' and first_user == False:
            # we've come to the next user turn
            #write previous user's row before overwriting it with this users data
            dialogue_eval_df.loc[user_turn_index] = [doc_id, dial_id, rc_id, turn_id, user_utterance, user_sps, agent_sps]
            #remember this user's data
            user_utterance = turn['utterance']
            turn_id = turn['turn_id']
            user_sps = [ref['sp_id'] for ref in turn['references']]
            dial_id = dialogues_for_doc.iloc[dialogue_index]['dial_id']
            rc_id = dial_id + '_'+ str(turn_id)
            #reset for next row
            agent_sps = []
            user_turn_index += 1
        elif role == 'agent':
            #gather agent spans but don't write the row just yet
            agent_sps.extend([ref['sp_id'] for ref in turn['references']])

#write the last line to the dataframe if the last dialogue's turn ended with an agent
if role == 'agent':
    dialogue_eval_df.loc[user_turn_index] = [doc_id, dial_id, rc_id, turn_id, user_utterance, user_sps, agent_sps]


dialogue_eval_df.sample(n=5)

,doc_id,dial_id,rc_id,turn_id,user_utterance,user_sp_ids,agent_sp_ids
44,Co-sign Your Spouse's Income-Driven Repayment ...,cf3cd3b2e35899f95cbe65913300891a,cf3cd3b2e35899f95cbe65913300891a_5,5,What if I do not have a reference number to co...,"[6, 7, 8]","[6, 7, 8]"
28,Co-sign Your Spouse's Income-Driven Repayment ...,ffe6e4350a3783cf449839b8e96a35bc,ffe6e4350a3783cf449839b8e96a35bc_9,9,and what should i do to start?,[15],[15]
18,Co-sign Your Spouse's Income-Driven Repayment ...,475064ee437d82e49a614ac9fc30f934,475064ee437d82e49a614ac9fc30f934_1,1,who is the co-signer on a repayment plan?,[1],"[2, 3]"
45,Co-sign Your Spouse's Income-Driven Repayment ...,cf3cd3b2e35899f95cbe65913300891a,cf3cd3b2e35899f95cbe65913300891a_7,7,Are there directions to access the reference n...,[9],[9]
14,Co-sign Your Spouse's Income-Driven Repayment ...,e41e73310cd72adc18ef0d4500bed86f,e41e73310cd72adc18ef0d4500bed86f_5,5,What if the applicant completed an IDR plan re...,[10],[11]


In [8]:
#some manual playing

# manual_test = 'What plate combinations are restricted?'
#manual_test = 'How much do personalized plates cost?'
# manual_test = 'Can I use personalized plates on any type of vehicle?'
manual_test = 'II\'m going to be co-signing a federal aid loan for my wife and I need some more information about how it works.'

#same preprocessing
manual_test_doc = simple_preprocess(manual_test, deacc=True)
vector = model.infer_vector(manual_test_doc)
sims = model.dv.most_similar([vector], topn=len(model.dv))
sims_dic = dict(sims)

labeled_results = dialogue_eval_df.loc[dialogue_eval_df['user_utterance'] == manual_test]
gold_user_sps = list(labeled_results['user_sp_ids'])[0]
gold_agent_sps = list(labeled_results['agent_sp_ids'])[0]

print(
    f'Labeled data result for user utterance "{manual_test}": user sp ids: {gold_user_sps} and following agent spans: {gold_agent_sps}\n')

print('Model predictions:\n')
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('THIRD-MOST', 2), ('FOURTH-MOST', 3), ('MEDIAN', len(sims) // 2),
                     ('LEAST', len(sims) - 1)]:
    #remember that the indexing of the raining doc starts at 0 but the one of the sp_ids at 1!!!!!
    training_doc = training_docs[sims[index][0] - 1]
    print(u'%s %s: %s «%s»' % (label, sims[index], training_doc.tags, ' '.join(training_doc.words)))

print('\nSpans that should have been selected based on gold labels are:')
for gold_label in gold_user_sps:
    gi = int(gold_label)
    #carefull with using tag to look up in list as lists start with 0 index and tags with 1!!!!
    training_doc = training_docs[gi - 1]
    text = ' '.join(training_doc.words)
    tag = training_doc.tags
    print(f'User label {gold_label} ({gi}, {sims_dic[gi]}): {tag} «{text}»')

for gold_label in gold_agent_sps:
    gi = int(gold_label)
    #carefull with using tag to look up in list as lists start with 0 index and tags with 1!!!!
    training_doc = training_docs[gi - 1]
    text = ' '.join(training_doc.words)
    tag = training_doc.tags
    print(f'Agent label {gold_label} ({gi}, {sims_dic[gi]}): {tag} «{text}»')


Labeled data result for user utterance "II'm going to be co-signing a federal aid loan for my wife and I need some more information about how it works.": user sp ids: ['1'] and following agent spans: ['2', '3']

Model predictions:

MOST (13, 0.7868408560752869): [13] «he or she will click on direct consolidation loan applications under my loan documents to locate and provide you with the co sign code you need»
SECOND-MOST (14, 0.750001847743988): [14] «co sign income driven repayment idr plan request»
THIRD-MOST (6, 0.7490072250366211): [6] «contact the idr applicant»
FOURTH-MOST (15, 0.692875862121582): [15] «log in to start»
MEDIAN (5, 0.12346689403057098): [5] «if you do not have the reference number co sign code»
LEAST (0, -0.4702322483062744): [26] «financial information note only if you file income taxes separately from your spouse»

Spans that should have been selected based on gold labels are:
User label 1 (1, 0.5984863042831421): [1] «what is co signer»
Agent label 2 (2, -0.29

#  Manual Evaluation


In [9]:
#Figure out how to deal with the squad_v2 metrics
from datasets import load_metric

metric = load_metric("squad_v2")
print(metric.features)

prediction = {'id': '0',
              'prediction_text': 'contact the idr applicant',
              'no_answer_probability': 0.0}
reference = {'id': '0', 'answers': {'text': ['what is co signer co singer is the spouse of an applicant who initiated an income driven repayment plan request as a co singer you are not obligated to repay this loan by signing borrower idr application '
                                        ],
                                    'answer_start': [1]}}
metric.add(prediction=prediction, reference=reference)
final_score = metric.compute()
final_score

{'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None), 'no_answer_probability': Value(dtype='float32', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}


{'exact': 0.0,
 'f1': 11.11111111111111,
 'total': 1,
 'HasAns_exact': 0.0,
 'HasAns_f1': 11.11111111111111,
 'HasAns_total': 1,
 'best_exact': 0.0,
 'best_exact_thresh': 0.0,
 'best_f1': 11.11111111111111,
 'best_f1_thresh': 0.0}

# Simplest prediction using rc dataset questions

In [11]:
# load the rc_dataset
rc_dataset = load_dataset(
    "doc2dial",
    name="doc2dial_rc",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)
import pandas as pd

rc_dataset_df = pd.DataFrame(data=rc_dataset)
rc_dataset_df.sample(n=5)



Reusing dataset doc2dial (./data_cache_doc2vec_notebook/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


,id,title,context,question,answers,domain
10908,3b0686e711a17da81d614a0d676f5a7a_4,Change Your VA Direct Deposit Information | Ve...,\n\nChange your VA direct deposit information ...,user:Can I change the type of account I have t...,{'text': ['you'll need a Premium DS Logon acco...,va
12492,48cd2b341259439812cee811c69fef97_14,VA Pension Benefits | Veterans Affairs#1_0,\n\nVA pension benefits \nVA pension benefits ...,user:is there a number for the online help? \t...,{'text': ['Support for current pension benefic...,va
15195,1a0705c2a86bb3ae2734bc10f383808f_4,Reserve Educational Assistance Program (REAP) ...,\n\nReserve Educational Assistance Program (RE...,user:yes \tagent:How does it affect me that RE...,{'text': ['Find the description below that bes...,va
12872,73ffd9b1e0ea5291d12c45c8f253d9ec_4,Montgomery GI Bill Active Duty (MGIB-AD) | Vet...,\n\nMontgomery GI Bill Active Duty (MGIB-AD) \...,"user:Yes, I have done some research and I thin...",{'text': ['you ll need to decide which to rece...,va
7710,bf48f124429123164ef5c509c6c8df7e_10,Disability Benefits | Social Security Administ...,\n\nDisability Benefits \nLearn about Disabili...,user:What information will I need to give? \ta...,{'text': ['Your date and place of birth and So...,ssa


In [12]:
# filter rc dataset to only contain the doc I've trained with
# using the doc id to filter

rc_dataset_doc_id = rc_dataset.filter(lambda doc: doc['title'] == doc_id)
rc_dataset_doc_id

  0%|          | 0/21 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'domain'],
    num_rows: 40
})

In [15]:
#now lets try to use the rc dataset to do the predictions
#simple infer vector for the question and use the most frequent response as response
predictions = []
references = []
print_number_of_examples = 4
for example in rc_dataset_doc_id:
    question_ = example["question"]
    # it does better if the user and agent string are left
    # question_ = question_.replace('user:', '')
    # question_ = question_.replace('agent:', '')

    #preprocess question in the same way
    test_doc = simple_preprocess(question_, deacc=True)
    #calculate vector using model
    vector = model.infer_vector(test_doc)
    #find the most similar document (spans)
    sims = model.dv.most_similar([vector], topn=1)
    most_likely_answer = training_docs[sims[0][0] - 1]
    most_likely_predicted_tag = most_likely_answer.tags
    #find original text for tag
    most_likely_predicted_text = spans[most_likely_predicted_tag[0]-1]
    if print_number_of_examples > 0:
        print_number_of_examples -= 1
        print(f'Question: {question_}\n')
        print(f'processed words: {most_likely_answer.words}\n')
        print(f'Original words: {most_likely_predicted_text}\n')

    id_ = example["id"]
    predictions.append(
        {'id': id_,
         'prediction_text':
             most_likely_predicted_text,
         'no_answer_probability': 0.0
         }
    )

    #just using their answers
    references.append(
        {
            "id": id_,
            "answers": example["answers"],
        }
    )

metric.add_batch(predictions=predictions, references=references)
final_score = metric.compute()
final_score


Question: user:II'm going to be co-signing a federal aid loan for my wife and I need some more information about how it works.

processed words: ['log', 'in', 'to', 'start']

Original words: LOG IN TO START 

Question: user:Does it matter that the Income-Driven Repayment Plan Request was done as part of a Direct Consolidation Loan Application? 	user:No. I'm not sure how it works. 	agent:Are you aware of your obligations to repay as the co-signer? 	agent:I can help you with that. A co-signer is the spouse of an applicant who initiated an Income-Driven Repayment Plan Request.  	user:II'm going to be co-signing a federal aid loan for my wife and I need some more information about how it works.

processed words: ['contact', 'the', 'idr', 'applicant']

Original words: contact the IDR applicant. 

Question: user:Who all can co-sign for this? 	agent:No. Your wife will just have to click on "Direct Consolidations Loan Applications" under "My Loan Documents" to find the co-sign code, you will n

{'exact': 7.5,
 'f1': 18.575694774800162,
 'total': 40,
 'HasAns_exact': 7.5,
 'HasAns_f1': 18.575694774800162,
 'HasAns_total': 40,
 'best_exact': 7.5,
 'best_exact_thresh': 0.0,
 'best_f1': 18.575694774800162,
 'best_f1_thresh': 0.0}

In [ ]:

d_id_reference = {}
for ex in rc_dataset:
    d_id_reference[ex["id"]] = 0
    references.append(
        {
            "id": ex["id"],
            "answers": ex["answers"],
        }
    )
assert (
        len(references)
        == len(d_id_reference)
), "Ensure the matching count of instances of references and predictioins"

references

# Questions

- How to evaluate an F1 if it's not a classifier?
- What data set should we use to train and which one to test?
- Evaluation why does the rc data set not have all the questions from the dialogues?
- Many mistakes are due to spans being so small, what was the sense behind that and do we have to  use their spans or would
it make more sense to keep a paragraph together?

- how to improve training e.g should I train accross all documents
- improve accross domain?

